# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#read the weather data into a dataframe
city_weather = pd.read_csv("Output_Data/city_weather.csv")

#display dataframe
city_weather.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Nanchang,28.6833,115.8833,69.40,55,7,7.23,CN,1635133027
1,Kapaa,22.0752,-159.3190,85.98,85,40,3.00,US,1635133027
2,Port Elizabeth,-33.9180,25.5701,52.09,93,0,5.75,ZA,1635133028
3,Atuona,-9.8000,-139.0333,78.26,71,5,18.48,PF,1635133028
4,Fort Collins,40.5853,-105.0844,55.08,43,1,1.01,US,1635132783


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
#heatmap of the humidity
locations = city_weather[["Lat", "Lng"]]
humidity = city_weather["Humidity"]
fig = gmaps.figure()

#add the heatmap layer to the figure
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_weather = city_weather.loc[
    (city_weather["Max Temp"] > 70) & (city_weather["Max Temp"] < 80) & \
    (city_weather["Wind Speed"] < 10) & (city_weather ["Cloudiness"]==0)
    
].dropna()

ideal_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
129,Port Augusta,-32.5000,137.7667,76.71,15,0,4.56,AU,1635133072
181,Saint-Pierre,-21.3393,55.4781,76.57,72,0,3.44,RE,1635133091
185,Salalah,17.0151,54.0924,73.49,60,0,5.75,OM,1635133092
301,Arlit,18.7369,7.3853,78.12,14,0,1.54,NE,1635133137
364,Doha,25.2867,51.5333,78.78,84,0,1.14,QA,1635132930
398,Dezful,32.3811,48.4058,72.30,23,0,1.77,IR,1635133172


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#create hotel dataframe
hotel_df = ideal_weather[["City", "Country", "Lat", "Lng"]]

#add "hotel name" column
hotel_df["Hotel Name"] = ""

#display dataframe
hotel_df

<ipython-input-6-dece8eac0e2e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Hotel Name
129,Port Augusta,AU,-32.5000,137.7667,
181,Saint-Pierre,RE,-21.3393,55.4781,
185,Salalah,OM,17.0151,54.0924,
301,Arlit,NE,18.7369,7.3853,
364,Doha,QA,25.2867,51.5333,
398,Dezful,IR,32.3811,48.4058,


In [7]:
#set the parameters to search for hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

#loop through the dataframe using iterrows()
for index, row in hotel_df.iterrows():
    #get the lat and lng from the current row
    lat = row["Lat"]
    lng = row["Lng"]

    #use the lat and lng to set the location value in the params
    params["location"] = f"{lat},{lng}"
    
    #use the googlee apis places base url to search
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    #make request and print the url
    name_address = requests.get(base_url, params=params).json()
    
    try:
        #populate the hotel name column
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except:
        print(f"Error finding lodging in {row['City']} - skipping")

hotel_df

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,City,Country,Lat,Lng,Hotel Name
129,Port Augusta,AU,-32.5000,137.7667,Majestic Oasis Apartments
181,Saint-Pierre,RE,-21.3393,55.4781,Lindsey Hôtel
185,Salalah,OM,17.0151,54.0924,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
301,Arlit,NE,18.7369,7.3853,Case de Passage De la Prefecture
364,Doha,QA,25.2867,51.5333,Movenpick Hotel Doha
398,Dezful,IR,32.3811,48.4058,سالن پیرایش امید


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))